# Registering For Verde Data 

This notebook provides a walk through the registration process to obtain Verde data for a field through the Agrimetrics Graph Query API.

Registering a field for Verde data requires the following inputs:

* Field ID: Thsi is the ID of the field of interest.

* Season: This is the growing season for which crop observations are of interest. Currently supported seasons include `SEP2017TOSEP2018`, `SEP2018ToSEP2019` and `SEP2019ToSEP2020`

* Type of layer: The layer type is either `CROP_SPECIFIC` or `NON_CROP_SPECIFIC`. `CROP_SEPCIFIC` would require crop type tp be specfied in the request.
    `CROP_SPECIFIC` provides measurements for two parameters: Leaf Area Index (LAI) and Chlorophyll Contents.
    `NON_CROP_SPECFIC` provides measurements for following parameters:
        * Brown Vegetation Cover Fraction
        * Canopy Foliage Mean Angle
        * Canopy Holes
        * Canopy Shadow Factor
        * Canopy Wetness Vertical Balance
        * Chlorophyll Content
        * Flower Storey Cover Fraction
        * Fraction Of Absorbed Photosynthetically Active Radiation
        * Green Vegetation Cover Fraction
        * Leaf Brown Index
        * Leaf Water Content
        * Normalised Difference Vegetation Index
        * Soil Fraction
        * Soil Water Saturation Fraction
                
* Crop Type: This is the sown crop for the season of interest. Currently supported crops include Corn, Wheat, Barley, Soya, Sugarbeet, Alomods,Rapeseed, Rice, Sunflower, Potatoes, Cotton, Alfalfa and Grass.

**There are two steps in a registration process:**

* Find out how much it costs to register a field for Verde data.
* Register the field for Verde data.

It is a highly recommended to find out about the cost of registering a field of interest using Step 1 before registering. The field ID used in these examples is a trial field in Reading, UK.

First, we import the necessary Python modules.

In [1]:
import os
import pandas as pd
import requests

Define the GraphQL endpoint and the API subscription key.

In [2]:
GRAPHQL_ENDPOINT = "https://api.agrimetrics.co.uk/query-api/v1/graphql"
if "API_KEY" in os.environ:
    API_KEY = os.environ["API_KEY"]
else:
    API_KEY = input("Query API Subscription Key: ").strip()
    
common_headers = {
    "OCP-APIM-Subscription-Key": API_KEY,
    "Accept": "application/json",
    "Content-Type": "application/json"
}

## Step 1: Find out how much it costs to register a field for Verde data.
 
We will first find out how much it costs to register our trial field to obtain Verde data.

Firstly we create variables object with the variables that we will use to get the costs for a specific registration

In [3]:
cost_variables = {
    "fieldId": "https://data.agrimetrics.co.uk/fields/-Dzkwq1lmP0VkSfEcpSqsQ",
    "season": "SEP2019TOSEP2020",
    "layerType": "CROP_SPECIFIC",
    "cropType": "GRASS"
}

Then we create a registration query

In [4]:
cost_query = """
    query RegistrationCostQuery(
        $fieldId: ID!,
        $layerType: LayerType!,
        $season: CropSeason!,
        $cropType: ObservationCropType
        ) 
    {
      account {
        premiumData {
          cropObservationRegistrationsCost(registrations: {
            fieldId: $fieldId, 
            layerType: $layerType , 
            season: $season, 
            cropType: $cropType}) {
            cost
          }
        }
      }
    }
"""

Construct the query object to pass to the API. 
Retrieve the data from the API using `requests`, and parse the returned JSON data.

In [5]:
cost_query_object = {
  "query": cost_query,
  "variables": cost_variables,
  "operationName": "RegistrationCostQuery"
}
result = requests.post(GRAPHQL_ENDPOINT, headers=common_headers, json=cost_query_object)
result.raise_for_status()
data = result.json()["data"]

Cost of registering a field is obtained from API response as follows

In [6]:
cost = data['account']['premiumData']['cropObservationRegistrationsCost'][0]['cost']

In [7]:
print(f"Cost of registering the field {cost_variables['fieldId']} for verde observations is £{cost}")

Cost of registering the field https://data.agrimetrics.co.uk/fields/-Dzkwq1lmP0VkSfEcpSqsQ for verde observations is £0


## Step 2: Registering a field for Verde data.
 
After finding the cost of registration we will register our trial field to obtain Verde data. We will use the same query variables in this step.

In [8]:
registration_query = """
    mutation RegistrationQuery($fieldId: ID!,
        $layerType: LayerType!,
        $season: CropSeason!,
        $cropType: ObservationCropType) {
      account {
        premiumData {
          addCropObservationRegistrations(registrations: {
            fieldId: $fieldId, 
            layerType: $layerType, 
            season: $season, 
            cropType: $cropType}) {
            cost
            id
          }
        }
      }
    }
"""

Construct the query object to pass to the API. Retrieve the data from the API using requests, and parse the returned JSON data.

In [9]:
registration_query_object = {
  "query": registration_query,
  "variables": cost_variables,
  "operationName": "RegistrationQuery"
}
result = requests.post(GRAPHQL_ENDPOINT, headers=common_headers, json=registration_query_object)
result.raise_for_status()
data = result.json()["data"]

Registering ID is obtained from API response as follows

In [10]:
registrationID = data['account']['premiumData']['addCropObservationRegistrations'][0]['id']

## Step 3: Getting Verde data for the registered field.
 
After registration we can query the Verde data.

In [11]:
verde_data_query = """
query getFieldIdAtLocation($fieldId: [ID!]!) {
  fields(where: {id: {EQ: $fieldId}}) {
    cropObservations {
      brownVegetationCoverFraction {
        unit
        median
        mean
      }
    }
  }
}

"""

In [12]:
data_query_object = {
  "query": verde_data_query,
  "variables": cost_variables,
  "operationName": "getFieldIdAtLocation"
}
result = requests.post(GRAPHQL_ENDPOINT, headers=common_headers, json=data_query_object)
result.raise_for_status()
data = result.json()["data"]

In [13]:
first_field = data["fields"][0]
brown_vegetation = first_field["cropObservations"]["brownVegetationCoverFraction"]
brown_vegetation_df = pd.DataFrame(brown_vegetation)

print(brown_vegetation_df)

                                                  unit    median      mean
0    http://data.agrimetrics.co.uk/units/decimal-fr...  0.126743  0.127750
1    http://data.agrimetrics.co.uk/units/decimal-fr...  0.126743  0.127750
2    http://data.agrimetrics.co.uk/units/decimal-fr...  0.131108  0.138733
3    http://data.agrimetrics.co.uk/units/decimal-fr...  0.131108  0.138733
4    http://data.agrimetrics.co.uk/units/decimal-fr...  0.101413  0.108096
..                                                 ...       ...       ...
149  http://data.agrimetrics.co.uk/units/decimal-fr...  0.021531  0.042932
150  http://data.agrimetrics.co.uk/units/decimal-fr...  0.134236  0.140376
151  http://data.agrimetrics.co.uk/units/decimal-fr...  0.134236  0.140376
152  http://data.agrimetrics.co.uk/units/decimal-fr...  0.102420  0.100937
153  http://data.agrimetrics.co.uk/units/decimal-fr...  0.102420  0.100937

[154 rows x 3 columns]
